- Extract wdate and reference

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT * FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT * FROM head', conn)
conn.close()
head = df1.append(df2)

In [2]:
df = pd.read_excel('KINT_var.xlsx')
words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first') # 신어에 해당하는 것 가져오기
df = head

In [3]:
import re
from string import punctuation

# Stopwords 처리
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
pattern2 = re.compile(r'[^가-힣 ]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

df['head'] = df['head'].map(lambda x: pattern3.sub(' ',
                                                       pattern2.sub('',
                                                                    pattern1.sub('', x))))

In [4]:
word3 = words.iloc[200:220]

In [ ]:
from collections import defaultdict
from tqdm import tqdm_notebook

# 날짜와 reference 가져오기
sent = {w:[(df.iloc[_]['wdate'], df.iloc[_]['ref']) for _ in range(len(df)) if w in df.iloc[_]['head']] for w in tqdm_notebook(word3)}
# Excel에 저장
pd.DataFrame.from_dict(sent, orient='index').to_excel('word3_1.xlsx')

C:\Users\gozj3\anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


- Clear Up Data

In [ ]:
# Data 원하는 형태로 가져오기
import ast
temp = pd.read_excel('wdate.xlsx', index_col='Unnamed: 0')
temp = temp.T.to_dict('list')
temp = {k:[ast.literal_eval(_) for _ in v if type(_) is str] for k,v in temp.items()}

In [ ]:
from collections import defaultdict
import re
# date별, ref별로 dict 만들기
date = defaultdict(lambda:0)
ref = defaultdict(lambda:0)
for k, v in temp.items():
    dtemp = defaultdict(lambda:0)
    rtemp = defaultdict(lambda:0)
    for _ in v:
        if re.search(r'\d+[.]\d+',_[0]):
            if len(re.sub(r'[.]', r'-', re.search(r'\d+[.]\d+',_[0]).group())) < 7:
                dtemp['20'+re.sub(r'[.]', r'-', re.search(r'\d+[.]\d+',_[0]).group())] += 1
            else:
                dtemp[re.sub(r'[.]', r'-', re.search(r'\d+[.]\d+',_[0]).group())] += 1 # format 통일
        elif re.search(r'\d+[-]\d+',_[0]):
            dtemp[re.search(r'\d+[-]\d+',_[0]).group()] += 1
        elif re.search(r'\d+년 \d+',_[0]):
            dtemp[re.sub(r'년 ', r'-', re.search(r'\d+년 \d+',_[0]).group())] += 1 # format 통일
        elif re.search(r'\d+[/]\d+[/]', _[0]):
            if len(re.sub(r'[/]', r'-', re.search(r'\d+[/]\d+[/]',_[0]).group())) < 7:
                dtemp['20'+re.sub(r'[/]', r'-', re.search(r'\d+/\d+/',_[0]).group())[:-1]] += 1 # format 통일
            else:
                dtemp[re.sub(r'[/]', r'-', re.search(r'\d+[/]\d+[/]',_[0]).group())[:-1]] += 1
        elif re.search(r'\d+[/]\d+', _[0]):
            dtemp['2019'+'-'+re.search(r'\d+', _[0]).group()]+=1
        else:
            print(_[1])
            dtemp[_[0]] += 1
        rtemp[_[1]] += 1
    date[k] = dtemp
    ref[k] = rtemp

In [ ]:
import math
# nan 값이 있으면 sum이 무조건 nan으로 처리되는 것을 피하기 위해서 overriding
def sum(data):
    sum = 0
    for _ in data:
        if not math.isnan(_):
            sum += _
    return sum

In [ ]:
# ref별 비율 구하기
ref = pd.DataFrame.from_dict(ref, orient='index')
total_ref = defaultdict(lambda:0)
for _ in head['ref']:
    total_ref[_] += 1
    
for _ in ref:
    ref[_] = ref[_]/total_ref[_] # 각 빈도를 전체 수집 데이터로 나눈다.

for _ in ref.index:
    ref.loc[_] = ref.loc[_]/sum(ref.loc[_]) # 전체를 1로 맞추고, 각 비율을 구한다.
    
# 이 달의 키워드를 뽑기 위한 수치 추출
this_month = pd.DataFrame.from_dict(date, orient='index')[['2020-08']]
key = defaultdict(lambda:0)
for _ in this_month.index:
    temp = list()
    for i in range(len(pd.DataFrame.from_dict(total_ref, orient='index'))):
        temp.append(pd.DataFrame.from_dict(total_ref, orient='index').iloc[i].values*pd.DataFrame(ref.loc[_]).iloc[i].values)
    temp = sum(temp)
    key[_] = this_month.loc[_].values/temp

In [ ]:
pd.DataFrame.from_dict(date, orient='index')[['2020-08', '2020-07', '2020-06', '2020-05', '2020-04', '2020-03','2020-02','2020-01','2019-12', '2019-11','2019-10','2019-09','2019-08']].to_excel('word4_date.xlsx') # 1년치만 가져온다.
ref.to_excel('word4_ref.xlsx')
pd.DataFrame.from_dict(key, orient='index').to_excel('word4_key.xlsx')